In [1]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()


In [400]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [2]:
# preprocessed file from Thomas
transfers_events = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export.csv')

/tmp/ipykernel_2052149/4215361439.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  transfers_events = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export.csv')


In [ ]:
# bunch of nan rows
transfers_events = transfers_events[~transfers_events['stay_id'].isna() & ~transfers_events['hadm_id'].isna()]


In [ ]:
# grab subject IDs
transfers_events = transfers_events.merge(estays[['stay_id', "subject_id"]], on="stay_id", how="left")

In [114]:
# discharge summaries
discharges = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge.csv.gz")

# ed stays
edstays = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/edstays.csv.gz')

# triage
triage = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/triage.csv.gz')

In [116]:
triage.stay_id.nunique(), triage.subject_id.nunique(), triage.shape

(71994, 48915, (71994, 11))

In [67]:
# ward transfers
transfers = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/transfers.csv.gz')

# higher-level services (ICU, CARD, etc)
services = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/services.csv.gz')

# get patient info
pts = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/patients.csv.gz')

# admission demographics
admissions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/admissions.csv.gz')

# procedures
procs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/procedures_icd.csv.gz')
procs_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_procedures.csv.gz')

# diagnoses
diags = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/diagnoses_icd.csv.gz')
diags_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_diagnoses.csv.gz')

# meds
med_orders = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/emar.csv.gz')


In [ ]:
med_orders = med_orders[~med_orders['charttime'].isna() & 
                        ~med_orders['medication'].isna() & 
                        ~med_orders['event_txt'].isna()]

In [ ]:
procs = procs.astype({"chartdate":"datetime64[ns]"})
med_orders = med_orders.astype({"charttime":"datetime64[ns]",
                  "scheduletime":"datetime64[ns]",
                  "storetime":"datetime64[ns]",})

discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

In [69]:
# are there multiple icd code mappings? 
procs_icd.shape, procs_icd.icd_code.nunique()
# looks like only for the two versions of ICD

((85257, 3), 85251)

In [70]:
procs_icd[procs_icd['icd_code'] == "067"]

,icd_code,icd_version,long_title
20005,067,9,Excision of thyroglossal duct or tract
20006,067,10,"Lower Veins, Dilation"


In [71]:
# drop any potential repeats
procs_icd = procs_icd.groupby(["icd_code", "icd_version"]).first().reset_index()
diags_icd = diags_icd.groupby(["icd_code", "icd_version"]).first().reset_index()

In [84]:
# grab long_titles for procs/diags
procs = procs.merge(procs_icd, on=["icd_code", "icd_version"], how="left")
diags = diags.merge(diags_icd, on=["icd_code", "icd_version"], how="left")

In [22]:

# why would someone have multiple ED stays for the same admission?
# looks like they came and left the ED multiple times during the course of their hospital course? 
transfers_events[transfers_events['hadm_id'] == 23880210].iloc[0]['careunit']

"['Emergency Department', 'Emergency Department', nan, 'Medicine']"

In [42]:
transfers[transfers['hadm_id'] == 29999828].sort_values("intime")[['intime',"outtime",'careunit']]

,intime,outtime,careunit
1242364,2180-10-27 05:24:15,2180-10-27 19:50:31,Discharge Lounge
1242365,2180-10-27 19:50:31,2180-10-28 09:54:07,Med/Surg
1242363,2180-10-28 09:54:07,2180-10-29 16:32:32,Coronary Care Unit (CCU)
1242361,2180-10-29 16:32:32,2180-10-30 17:27:11,Med/Surg
1242362,2180-10-30 17:27:11,NaN,NaN


In [17]:
# transfers_events.stay_id.duplicated().sum()
transfers_events.hadm_id.duplicated().sum()

159

In [352]:
# create initial input text prompt
def get_demos(subject_id):
    # has gender, anchor-age, date-of-death if exists
    return pts[pts['subject_id'] == subject_id].squeeze()
    
def get_transfers(hadm_id):
    return transfers[transfers['hadm_id'] == hadm_id].sort_values("intime").squeeze()
    
def get_procs_within_service(hadm_id, transfer_event):
    adm_procs = procs[procs['hadm_id'] == hadm_id]
    adm_procs = adm_procs.sort_values("seq_num")
    
    adm_procs_in_unit = adm_procs[(adm_procs['chartdate'] > transfer_event['intime'])
                                & (adm_procs['chartdate'] < transfer_event['outtime'])]
    return adm_procs_in_unit
    
def get_procs(hadm_id):
    adm_procs = procs[procs['hadm_id'] == hadm_id]
    return adm_procs.sort_values("seq_num")

def get_diags(hadm_id):
    adm_diags = diags[diags['hadm_id'] == hadm_id]
    return adm_diags.sort_values("seq_num")

def get_med_orders_within_service(hadm_id, transfer_event):
    med_admin = med_orders[(med_orders['hadm_id'] == hadm_id) & (med_orders['event_txt'] == "Administered")]
    med_admin = med_admin.sort_values("emar_seq")
    
    adm_diags_in_unit = med_admin[(med_admin['charttime'] > transfer_event['intime'])
                                & (med_admin['charttime'] < transfer_event['outtime'])]

    adm_diags_in_unit['admin_text'] = adm_diags_in_unit['medication'] + " at " + adm_diags_in_unit['charttime'].dt.strftime('%B %d, %Y, %r')
    
    return adm_diags_in_unit

def get_med_orders(hadm_id):
    med_admin = med_orders[(med_orders['hadm_id'] == hadm_id) & (med_orders['event_txt'] == "Administered")]
    med_admin['admin_text'] = med_admin['medication'] + " at " + med_admin['charttime']
    return med_admin

def get_triage_info(stay_id):
    return triage[triage['stay_id'] == stay_id]

In [354]:
def create_pt_prompt(discharge_row):

    demos = get_demos(discharge_row['subject_id'])
    if demos.empty:
        age = r"[UNKNOWN AGE]"
        sex = r"[UNKNOWN SEX]"
    else:
        age = demos['anchor_age']
        sex = demos['gender']
    
    chief_complaints = discharge_row['chiefcomplaint']
    if sex:
        pronoun = ["he","his"] if sex == "M" else ['she',"her"]
    else:
        pronoun = ["they", "their"]

    careunits = ast.literal_eval(discharge_row['careunit'].replace('nan', 'None'))
    careunits = [x for x in careunits if x]

    # transfers with dates
    tranfers = get_transfers(discharge_row['hadm_id'])
    
    procs = get_procs(discharge_row['hadm_id'])
    diags = get_diags(discharge_row['hadm_id'])
    
    med_orders = get_med_orders(discharge_row['hadm_id'])

    prompt = f"___ is a {age} year old {sex} presenting to the ED with {chief_complaints}. Over the course of {pronoun[1]} hospital course, ___ started at {careunits[0]} and then visited {', '.join(careunits[1:])}. Over the course of their hospital stay, {pronoun[0]} was given the following diagnoses: {', '.join(diags['long_title'])} in order of importance to this admission.\n\nThis led to receiving the following procedures: {', '.join(procs['long_title'])} in order of priority.\n\n___ also received the following medications: {', '.join(med_orders['admin_text'])}"
    
    return prompt
    


In [355]:
transfers[transfers['hadm_id'] == transfers['hadm_id'].sample(1).squeeze()].sort_values("intime")

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
1660873,18775358,25643992.0,37597238,ED,Emergency Department,2140-06-12 10:28:00,2140-06-12 17:21:48
1660872,18775358,25643992.0,37540703,admit,Med/Surg,2140-06-12 17:21:48,2140-06-12 18:56:01
1660874,18775358,25643992.0,37642842,transfer,Discharge Lounge,2140-06-12 18:56:01,2140-06-12 20:05:50
1660871,18775358,25643992.0,37250858,transfer,Med/Surg/GYN,2140-06-12 20:05:50,2140-06-12 22:29:37
1660869,18775358,25643992.0,33390977,transfer,Med/Surg/GYN,2140-06-12 22:29:37,2140-06-12 22:59:11
1660870,18775358,25643992.0,34975134,transfer,Med/Surg/GYN,2140-06-12 22:59:11,2140-06-15 00:14:52
1660875,18775358,25643992.0,38569653,transfer,PACU,2140-06-15 00:14:52,2140-06-15 04:10:27
1660877,18775358,25643992.0,39206091,transfer,Med/Surg/Trauma,2140-06-15 04:10:27,2140-06-17 17:06:56
1660876,18775358,25643992.0,38845120,discharge,NaN,2140-06-17 17:06:56,NaN


In [ ]:
print(create_pt_prompt(transfers_events.sample(1).squeeze()))

___ is a 74 year old M presenting to the ED with Seizure, Transfer. Over the course of his hospital course, ___ started at Neurology and then visited Emergency Department. Over the course of their hospital stay, his was given the following diagnoses: Cerebral infarction due to embolism of unspecified cerebral artery, Urinary tract infection, site not specified, Acute kidney failure, unspecified, Unspecified convulsions, Prediabetes, Essential (primary) hypertension, Cerebral atherosclerosis, Hyperlipidemia, unspecified, Occlusion and stenosis of left carotid artery, Atherosclerotic heart disease of native coronary artery without angina pectoris, Pure hypercholesterolemia, unspecified, Polyneuropathy, unspecified, Coronary angioplasty status, Patient's other noncompliance with medication regimen, Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits, Long term (current) use of antithrombotics/antiplatelets, Personal history of nicotine de

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
prompts = transfers_events.sample(500).progress_apply(create_pt_prompt, axis=1)

100%|██████████| 500/500 [12:29<00:00,  1.50s/it]


In [ ]:
transfers_events['prompts'] = prompts

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
transfers_events_with_prompts = transfers_events[~transfers_events['prompts'].isna()]

In [ ]:
# Output 

# transfers_events_with_prompts.astype({"hadm_id":"int"}).rename({"Unnamed: 0":"idx"}, axis=1)[["idx", 'hadm_id','brief_hospital_course','prompts']].to_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export_with_prompts.csv", index=False)
# transfers_events_with_prompts.astype({"hadm_id":"int"}).rename({"Unnamed: 0":"idx"}, axis=1)[["idx", 'hadm_id','brief_hospital_course','prompts']].to_html("combined_data_export_with_prompts.html", index=False)

In [ ]:
from tabulate import tabulate
tabulate(transfers_events_with_prompts.astype({"hadm_id":"int"}).rename({"Unnamed: 0":"idx"}, axis=1)[["idx", 'hadm_id','brief_hospital_course','prompts']], tablefmt="tsv")

'    7\t    7\t29149108\tMr. ___ was an ___ man with prior gangrenous cholecystitis and \ncholedocholithiasis c/b cholangitis s/p sphincterotomy and \ncholecystostomy tube and stent (stent removed ___ with \ncholecystectomy ___, hypertension, non-insulin dependent \ndiabetes, and other issues who had a recent admission for \nhepatic toxicity thought to be due to supplement use who is \npresenting with fatigue, admitted again from ___ for \nacute hepatitis of unclear etiology. \n\n# Acute Liver Failure\nMr. ___ presented to ___ on ___ for his second admission \nwith pruritis and fatigue and was found to have increasing LFTs \nat clinic visit. Workup so far without evidence of viral \nhepatitis B, C, A, CMV, EBV, toxoplasmosis, and negative ___, \nAMA, and ___.\n\nMr. ___ denied the use of over the counter meds or herbal \nsupplements and had not had any recent new medications. Liver \nbiopsy findings non-specific, showing hepatic fibrosis and \nbiliary injury which is possibly continued

# Service-Level Progress Note Generation 

This section generates a pseudo-progress note from service-level information using GPT-3. It uses the raw data, instead of the preprocessed transfers_events data above from Thomas. 


In [415]:
def create_pt_prompt_per_service(discharge_row):

    demos = get_demos(discharge_row['subject_id'])
    if demos.empty:
        age = r"[UNKNOWN AGE]"
        sex = r"[UNKNOWN SEX]"
    else:
        age = demos['anchor_age']
        sex = demos['gender']

    pt_edstays = edstays[edstays['hadm_id'] == discharge_row['hadm_id']]

    ccs = []
    for stay_id in pt_edstays['stay_id'].tolist():
        triage_info = get_triage_info(stay_id)
        ccs.append(triage_info['chiefcomplaint'].squeeze())
        
    chief_complaints = ", ".join(ccs)

    if sex:
        pronoun = ["he","his"] if sex == "M" else ['she',"her"]
    else:
        pronoun = ["they", "their"]

    # transfers with dates
    transfers = get_transfers(discharge_row['hadm_id'])

    # get stay admission diagnoses
    diags = get_diags(discharge_row['hadm_id'])

    init_prompt = f"___ is a {age} year old {sex} that initially presented to the ED with {chief_complaints}. By the end of {pronoun[1]} hospital stay, {pronoun[0]} was given the following diagnoses: {', '.join(diags['long_title'])} in order of importance to this admission. "
    transfer_service_prompts = []

    for index, row in transfers.iterrows():
        if row['eventtype'] == "discharge":
             transfer_service_prompts.append("")
        else:
            # TODO: add from here. 
            within_service_prompt = f"The patient was transferred and stayed in the {row['careunit']} ward between {row['intime']} and {row['outtime']}. ___ received "
            
            procs = get_procs_within_service(discharge_row['hadm_id'], row)
            med_orders = get_med_orders_within_service(discharge_row['hadm_id'], row)
    
            within_service_prompt = within_service_prompt + f"the following procedures (ordered by priority): {', '.join(procs['long_title'].tolist())}. "
            within_service_prompt = within_service_prompt + f"{pronoun[0]} also received the following medications (ordered chronologically) during the service: {', '.join(med_orders['medication'].tolist())}. "
            
            full_prompt = init_prompt + within_service_prompt + f"Given this information, please generate a progress note for this patient for their care during this part of their hospital course staying in the {row['careunit']} ward."
            transfer_service_prompts.append(full_prompt)

    transfers['service_prompts'] = transfer_service_prompts
    
    return transfers
    


In [416]:
discharges.sample(1)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
10661,11531320-DS-18,11531320,24186866,DS,18,2175-03-06,2175-03-07 00:25:00,\nName: ___ Unit No: ___\n ...


In [ ]:
service_prompts = create_pt_prompt_per_service(discharges.sample(1).squeeze())

In [430]:


openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
engine = "decile-gpt-35-turbo-16k"



message_text = [{"role":"system","content":f"You are a physician that is reviewing a patient's medical record during their stay in your ward: {service_prompts.iloc[1]['careunit']} and writing progress notes based on this information"},]

deprescribe_prompt = {"role":"user",
                 "content":service_prompts['service_prompts'].iloc[1]}


message_text.append(deprescribe_prompt)
print(f"Deprescribe Prompt: {message_text}")

completion = openai.ChatCompletion.create(
  engine=engine,
  messages = message_text,
)


Deprescribe Prompt: [{'role': 'system', 'content': "You are a physician that is reviewing a patient's medical record during their stay in your ward: Medicine and writing progress notes based on this information"}, {'role': 'user', 'content': '___ is a 47 year old F that initially presented to the ED with Wound eval. Over the course of their hospital stay, she was given the following diagnoses: Type 1 diabetes mellitus with other specified complication, Other acute osteomyelitis, left ankle and foot, Other chronic osteomyelitis, left ankle and foot, Type 1 diabetes mellitus with diabetic peripheral angiopathy with gangrene, Other primary thrombophilia, Type 1 diabetes mellitus with foot ulcer, Non-pressure chronic ulcer of other part of left foot with unspecified severity, Unspecified lump in the left breast, unspecified quadrant, Embolism due to vascular prosthetic devices, implants and grafts, subsequent encounter, Surgical operation with implant of artificial internal device as the c

In [431]:
print(completion['choices'][0]['message']['content'])

Patient Name: ___
Age: 47
Gender: Female

Date: [Date of the progress note]

Subjective:
The patient remains in stable condition during her stay in the Medicine ward. She reports no significant changes in her symptoms or overall well-being since her transfer. The pain from her foot ulcers and osteomyelitis is adequately managed with the prescribed pain medications.

Objective:
Vital Signs:
- Blood pressure: __/__ mmHg
- Heart rate: __ beats per minute
- Respiratory rate: __ breaths per minute
- Temperature: __°C
- Oxygen saturation: __%

General:
The patient appears comfortable and in no apparent distress. She is alert, oriented, and responsive. Skin is warm and dry, with no signs of cyanosis or pallor.

Primary Diagnosis:
1. Type 1 diabetes mellitus with other specified complications
2. Other acute osteomyelitis, left ankle and foot
3. Other chronic osteomyelitis, left ankle and foot
4. Type 1 diabetes mellitus with diabetic peripheral angiopathy with gangrene
5. Other primary thrombo